In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import gym
from IPython import display
import time as t
%matplotlib inline
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim
import numpy as np
import random

env = gym.make('CartPole-v1')

In [ ]:
episodes = 2000
observations = []
rewards = []


for episode in range(episodes) :
    initial_state = env.reset()
    ended = False
    rewardsum = 0
    while not ended :
        env.render()
        action = env.action_space.sample()
        obs, reward, ended, info = env.step(action)
        #print(env.observation_space)
        observations.append(obs)
        rewardsum += reward
        #print(rewardsum)
    t.sleep(5)

velocities = []
ang_velocities = []
for observation in observations :
    velocities.append(observation[1])
    ang_velocities.append(observation[3])

print (f'min velocity : {np.min(velocities)}, max velocity : {np.max(velocities)}\n')
print (f'min ang_velocity : {np.min(ang_velocities)}, max ang_velocity : {np.max(ang_velocities)}')
      
    



In [ ]:
rewards = []


for episode in range(episodes) :
    initial_state = env.reset()
    ended = False
    rewardsum = 0
    while not ended :
        env.render()
        action = env.action_space.sample()
        obs, reward, ended, info = env.step(action)
        #print(env.observation_space)
        observations.append(obs)
        rewardsum += reward
        #print(rewardsum)
    t.sleep(5)

velocities = []
ang_velocities = []
for observation in observations :
    velocities.append(observation[1])
    ang_velocities.append(observation[3])

print (f'min velocity : {np.min(velocities)}, max velocity : {np.max(velocities)}\n')
print (f'min ang_velocity : {np.min(ang_velocities)}, max ang_velocity : {np.max(ang_velocities)}')
      
    



In [ ]:
def mapStateSpace(obs):

    max_position = 2.4 
    max_velocity = 5   
    max_angle = np.pi   
    max_angular_velocity = 4  

    # Normalize and discretize 
    a = np.round((obs[0] / max_position) * 10)  # Discretize position into 10 bins
    b = np.round((obs[1] / max_velocity) * 15)  # Discretize velocity into 10 bins
    c = np.round((obs[2] / max_angle) * 36)     # Discretize angle into 18 bins (~20 degrees each)
    d = np.round((obs[3] / max_angular_velocity) * 15)  # Discretize angular velocity into 10 bins

    return int(10*a + 10*b + 30*c + 10*d)
   
    

In [ ]:

n_states = 4000  
n_actions = 2

Q_table = np.zeros((n_states, n_actions))

learning_rate = 0.1
discount_factor = 1
exploration_prob = 0.2
epochs = 1000

for epoch in range(epochs):

    current_state = env.reset()
    #env.render()
    ended = False
    rewardsum = 0


    while not ended :

        if np.random.rand() < exploration_prob:
            action = env.action_space.sample()  
        else:
            action = np.argmax(Q_table[mapStateSpace(current_state)])  
        
        next_state, reward, ended, info = env.step(action)

        next_max = np.max(Q_table[mapStateSpace(next_state)])
        td_target = reward + discount_factor * next_max
        td_error = td_target - Q_table[mapStateSpace(current_state), action]
        Q_table[mapStateSpace(current_state), action] += learning_rate * td_error
        rewardsum += reward

        current_state = next_state 
    #print(rewardsum)
    

print("Learned Q-table:")

    







In [ ]:
episodes = 2000
rewards = []


for episode in range(episodes) :
    state = env.reset()
    ended = False
    rewardsum = 0
    while not ended :
        env.render()
        action = np.argmax(Q_table[mapStateSpace(state)])
        state, reward, ended, info = env.step(action)
        rewardsum += reward
    #print(rewardsum)
    #t.sleep(1)
    rewards.append(rewardsum)

print(np.max(rewards))

In [ ]:
plt.plot(range(len(rewards)), rewards)

In [ ]:
class Model(nn.Module):

    def __init__(self, in_features=4, h1=64, h2=32, out_features=2) -> None:
        super().__init__()
        self.fc1 = nn.Linear(in_features,h1)
        self.fc2 = nn.Linear(h1,h2)
        self.out = nn.Linear(h2, out_features)


    def forward(self, x):
        x = F.relu(self.fc1(x))
        #x = F.dropout(x, 0.2)
        x = F.relu(self.fc2(x))
        x = self.out(x)
        return x

policy_network = Model()

In [ ]:
import math


def epsilon_decay(epsilon, t, epsilon_min, epsilon_decay):
    return max(epsilon_min, min(epsilon, 1.0 - math.log10((t+1) * epsilon_decay)))

In [ ]:
from collections import deque


learning_rate = 0.001
discount_factor = 0.9
exploration_prob = 1.0
min_exploration_prob = 0.01
decay = 0.999

epochs = 5000

replay_buffer_batch_size = 128
min_replay_buffer_size = 5000
replay_buffer = deque(maxlen=min_replay_buffer_size)

target_network = Model()
target_network.load_state_dict(policy_network.state_dict())


optimizer = torch.optim.Adam(policy_network.parameters(), learning_rate)

loss_function = nn.MSELoss()

rewards = []

losses = []

loss = -100

for i in range(epochs) :

    #print(exploration_prob)
    exploration_prob = epsilon_decay(exploration_prob, i, min_exploration_prob, decay)

    terminal = False

    if i % 30 == 0 :
        target_network.load_state_dict(policy_network.state_dict())

    current_state = env.reset()

    rewardsum = 0

    p = False

    while not terminal :

        #env.render()

        if np.random.rand() < exploration_prob:
            action = env.action_space.sample()  
        else:
            state_tensor = torch.tensor(np.array([current_state]), dtype=torch.float32)
            with torch.no_grad():
                q_values = policy_network(state_tensor)
            action = torch.argmax(q_values).item()
        
        next_state, reward, terminal, info = env.step(action)

        rewardsum+=reward

        replay_buffer.append((current_state, action, terminal, reward, next_state))

        if(len(replay_buffer) >= min_replay_buffer_size) :

            minibatch = random.sample(replay_buffer, replay_buffer_batch_size)

            batch_states = torch.tensor([transition[0] for transition in minibatch], dtype=torch.float32)
            batch_actions = torch.tensor([transition[1] for transition in minibatch], dtype=torch.int64)
            batch_terminal = torch.tensor([transition[2] for transition in minibatch], dtype=torch.bool)
            batch_rewards = torch.tensor([transition[3] for transition in minibatch], dtype=torch.float32)
            batch_next_states = torch.tensor([transition[4] for transition in minibatch], dtype=torch.float32)

            with torch.no_grad():
                q_values_next = target_network(batch_next_states).detach()
                max_q_values_next = q_values_next.max(1)[0] 

            y = batch_rewards + (discount_factor * max_q_values_next * (~batch_terminal))    

            q_values = policy_network(batch_states).gather(1, batch_actions.unsqueeze(-1)).squeeze(-1)

            loss = loss_function(y,q_values)

            losses.append(loss)

            optimizer.zero_grad()
            
            loss.backward()

            #torch.nn.utils.clip_grad_norm_(policy_network.parameters(), 10)

            optimizer.step()

        if i%100 == 0 and not p:
            print(loss)
            p = True

        current_state = next_state
        
        

    rewards.append(rewardsum)

    torch.save(policy_network, 'stick_game.pth')


In [ ]:

#plt.plot(range(len(losses)), [l.detach().numpy() for l in losses])

#print(losses[len(losses)-1])

torch.save(policy_network, 'stick_game.pth')



In [15]:
with torch.no_grad():
    episodes = 2000
    rewards = []


    for episode in range(episodes) :
        state = env.reset()
        ended = False
        rewardsum = 0
        #state, reward, ended, info = env.step(env.action_space.sample())
        #state, reward, ended, info = env.step(env.action_space.sample())
        while not ended :
            env.render()
            #state, reward, ended, info = env.step(env.action_space.sample())
            state_tensor = torch.tensor([state], dtype=torch.float32)
            q_values = policy_network(state_tensor)
            action = torch.argmax(q_values).item()

            state, reward, ended, info = env.step(action)
            rewardsum += reward
        print(rewardsum)
        #t.sleep(1)
        rewards.append(rewardsum)

print(np.max(rewards))
        

499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0
499.0


KeyboardInterrupt: 